In [12]:
import boto3
import logging
import warnings
import pandas as pd
from sys import stderr

In [13]:
s3 = boto3.client('s3')
warnings.filterwarnings("ignore")
logging.basicConfig(stream=stderr, level="INFO")

In [13]:
import os
import boto3
from io import BytesIO
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from io import BytesIO
import json

In [34]:

def decrypt_existing_encrypted_file(s3_key, bucket_name, key, aad):
    s3 = boto3.client('s3')

    # Define the path to the existing info.json file
    info_file_key = s3_key.replace('.csv', '_info.json')

    # Retrieve the existing info.json file
    try:
        info_obj = s3.get_object(Bucket=bucket_name, Key=info_file_key)
        info_data = json.loads(info_obj['Body'].read())
        nonce = bytes.fromhex(info_data['nonce'])  # Convert hex nonce back to bytes
        print(f"Successfully retrieved nonce from {info_file_key}")
    except ClientError as e:
        print(f"Failed to retrieve info.json at {info_file_key}: {e}")
        return

    # Fetch the encrypted file from S3
    try:
        encrypted_obj = s3.get_object(Bucket=bucket_name, Key=s3_key)
        encrypted_content = encrypted_obj['Body'].read()
    except ClientError as e:
        print(f"Failed to retrieve encrypted file at {s3_key}: {e}")
        return

    # Decrypt the file content
    aesgcm = AESGCM(key)
    try:
        decrypted_content = aesgcm.decrypt(nonce, encrypted_content, aad)
        print(f"Decryption successful for {s3_key}")
    except Exception as e:
        print(f"Decryption failed: {e}")
        return

    # Save decrypted content to a temporary file path or handle it as needed
    temp_file_path = f"/tmp/{s3_key.split('/')[-1]}"
    with open(temp_file_path, 'wb') as f:
        f.write(decrypted_content)
    print(f"Decrypted content saved to {temp_file_path}")

    # Return the path to the decrypted file
    return temp_file_path

In [35]:
bucket_name = 'otg-fraud-dataml-prod'
s3_key = 'acm/daily/year=2024/month=10/day=29/20241030131824_acm_output.csv'
key=b')\xd9;_kp\xd5\x9dY\xc6\xb2UD\xa4r\xe1}\xf3\xb0+C\xba\xb4W\xa1\xc8\x00m\x94\xe2y\xc2'
aad = b"authenticated additional data" 

In [36]:
decrypted_file=decrypt_existing_encrypted_file(s3_key, bucket_name, key, aad)

Successfully retrieved nonce from acm/daily/year=2024/month=10/day=29/20241030131824_acm_output_info.json
Decryption successful for acm/daily/year=2024/month=10/day=29/20241030131824_acm_output.csv
Decrypted content saved to /tmp/20241030131824_acm_output.csv


In [26]:
date_part = s3_key.split('/') # ['2024', '10', '2']
timestamp = date_part[-1].split('_')[0]
# Extract year, month, and day from the file path
date_part = s3_key.split('/')[-4:-1]  # ['2024', '10', '2']

year, month, day = date_part[0], date_part[1], date_part[2]

# Construct the S3 path
s3_output_path = f"s3://otg-fraud-dataml-prod/acm/daily/temporary/{year}/{month}/{day}/{timestamp}_acm_output.csv"

In [27]:
s3_output_path

's3://otg-fraud-dataml-prod/acm/daily/temporary/year=2024/month=10/day=29/20241030131824_acm_output.csv'

In [2]:
import boto3
import logging
import warnings
import pandas as pd
from sys import stderr

In [3]:
s3 = boto3.client('s3')
warnings.filterwarnings("ignore")
logging.basicConfig(stream=stderr, level="INFO")

In [13]:
import os
import boto3
from io import BytesIO
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from io import BytesIO
import json

In [34]:

def decrypt_existing_encrypted_file(s3_key, bucket_name, key, aad):
    s3 = boto3.client('s3')

    # Define the path to the existing info.json file
    info_file_key = s3_key.replace('.csv', '_info.json')

    # Retrieve the existing info.json file
    try:
        info_obj = s3.get_object(Bucket=bucket_name, Key=info_file_key)
        info_data = json.loads(info_obj['Body'].read())
        nonce = bytes.fromhex(info_data['nonce'])  # Convert hex nonce back to bytes
        print(f"Successfully retrieved nonce from {info_file_key}")
    except ClientError as e:
        print(f"Failed to retrieve info.json at {info_file_key}: {e}")
        return

    # Fetch the encrypted file from S3
    try:
        encrypted_obj = s3.get_object(Bucket=bucket_name, Key=s3_key)
        encrypted_content = encrypted_obj['Body'].read()
    except ClientError as e:
        print(f"Failed to retrieve encrypted file at {s3_key}: {e}")
        return

    # Decrypt the file content
    aesgcm = AESGCM(key)
    try:
        decrypted_content = aesgcm.decrypt(nonce, encrypted_content, aad)
        print(f"Decryption successful for {s3_key}")
    except Exception as e:
        print(f"Decryption failed: {e}")
        return

    # Save decrypted content to a temporary file path or handle it as needed
    temp_file_path = f"/tmp/{s3_key.split('/')[-1]}"
    with open(temp_file_path, 'wb') as f:
        f.write(decrypted_content)
    print(f"Decrypted content saved to {temp_file_path}")

    # Return the path to the decrypted file
    return temp_file_path

In [51]:
bucket_name = 'otg-fraud-dataml-prod'
s3_path = 's3://otg-fraud-dataml-prod/acm/daily/year=2024/month=10/day=29/20241030131824_acm_output.csv'
key=b')\xd9;_kp\xd5\x9dY\xc6\xb2UD\xa4r\xe1}\xf3\xb0+C\xba\xb4W\xa1\xc8\x00m\x94\xe2y\xc2'
aad = b"authenticated additional data" 

In [52]:
decrypted_file=decrypt_existing_encrypted_file(s3_path)

Successfully retrieved nonce from acm/daily/year=2024/month=10/day=29/20241030131824_acm_output_info.json
Decryption successful for acm/daily/year=2024/month=10/day=29/20241030131824_acm_output.csv
Decrypted content uploaded back to S3 at s3://otg-fraud-dataml-prod/acm/daily/temporary/year=2024/month=10/day=29/20241030131824_acm_output.csv


In [36]:
decrypted_file=decrypt_existing_encrypted_file(s3_key, bucket_name, key, aad)

Successfully retrieved nonce from acm/daily/year=2024/month=10/day=29/20241030131824_acm_output_info.json
Decryption successful for acm/daily/year=2024/month=10/day=29/20241030131824_acm_output.csv
Decrypted content saved to /tmp/20241030131824_acm_output.csv


In [26]:
date_part = s3_key.split('/') # ['2024', '10', '2']
timestamp = date_part[-1].split('_')[0]
# Extract year, month, and day from the file path
date_part = s3_key.split('/')[-4:-1]  # ['2024', '10', '2']

year, month, day = date_part[0], date_part[1], date_part[2]

# Construct the S3 path
s3_output_path = f"s3://otg-fraud-dataml-prod/acm/daily/temporary/{year}/{month}/{day}/{timestamp}_acm_output.csv"

In [27]:
s3_output_path

's3://otg-fraud-dataml-prod/acm/daily/temporary/year=2024/month=10/day=29/20241030131824_acm_output.csv'

In [41]:
def extract_s3_key(s3_value):
    """Extract the S3 key if the full S3 path is provided, otherwise assume it's already a key."""
    if s3_value.startswith('s3://'):
        # Extract the key from the full S3 path
        return '/'.join(s3_value.split('/')[3:])
    else:
        # It's already a key
        return s3_value

In [38]:
extract_s3_key('s3://otg-fraud-dataml-prod/acm/daily/temporary/year=2024/month=10/day=29/20241030131824_acm_output.csv')

'acm/daily/temporary/year=2024/month=10/day=29/20241030131824_acm_output.csv'

In [48]:
def decrypt_existing_encrypted_file(s3_path):
    s3 = boto3.client('s3')
    s3_key = extract_s3_key(s3_path)
    info_file_key = s3_key.replace('.csv', '_info.json')

    # Retrieve the existing info.json file
    try:
        info_obj = s3.get_object(Bucket=bucket_name, Key=info_file_key)
        info_data = json.loads(info_obj['Body'].read())
        nonce = bytes.fromhex(info_data['nonce'])  # Convert hex nonce back to bytes
        print(f"Successfully retrieved nonce from {info_file_key}")
    except ClientError as e:
        print(f"Failed to retrieve info.json at {info_file_key}: {e}")
        return

    # Fetch the encrypted file from S3
    try:
        encrypted_obj = s3.get_object(Bucket=bucket_name, Key=s3_key)
        encrypted_content = encrypted_obj['Body'].read()
    except ClientError as e:
        print(f"Failed to retrieve encrypted file at {s3_key}: {e}")
        return

    # Decrypt the file content
    aesgcm = AESGCM(key)
    try:
        decrypted_content = aesgcm.decrypt(nonce, encrypted_content, aad)
        print(f"Decryption successful for {s3_key}")
    except Exception as e:
        print(f"Decryption failed: {e}")
        return
        
    date_part = s3_path.split('/') # ['2024', '10', '2']
    timestamp = date_part[-1].split('_')[0]
    # Extract year, month, and day from the file path
    date_part = s3_path.split('/')[-4:-1]  # ['2024', '10', '2']
    
    year, month, day = date_part[0], date_part[1], date_part[2]
    
    # Construct the S3 path
    s3_output_path = f"s3://otg-fraud-dataml-prod/acm/daily/temporary/{year}/{month}/{day}/{timestamp}_acm_output.csv"
    # Upload the decrypted content back to S3
    try:
        s3.put_object(
            Bucket=bucket_name,
            Key=s3_output_path,  # Adjust the path if needed
            Body=decrypted_content
        )
        print(f"Decrypted content uploaded back to S3 at {s3_output_path}")
    except Exception as e:
        print(f"Failed to upload decrypted file: {e}")


In [2]:
import boto3
import logging
import warnings
import pandas as pd
from sys import stderr

In [3]:
s3 = boto3.client('s3')
warnings.filterwarnings("ignore")
logging.basicConfig(stream=stderr, level="INFO")

In [13]:
import os
import boto3
from io import BytesIO
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from io import BytesIO
import json

In [34]:

def decrypt_existing_encrypted_file(s3_key, bucket_name, key, aad):
    s3 = boto3.client('s3')

    # Define the path to the existing info.json file
    info_file_key = s3_key.replace('.csv', '_info.json')

    # Retrieve the existing info.json file
    try:
        info_obj = s3.get_object(Bucket=bucket_name, Key=info_file_key)
        info_data = json.loads(info_obj['Body'].read())
        nonce = bytes.fromhex(info_data['nonce'])  # Convert hex nonce back to bytes
        print(f"Successfully retrieved nonce from {info_file_key}")
    except ClientError as e:
        print(f"Failed to retrieve info.json at {info_file_key}: {e}")
        return

    # Fetch the encrypted file from S3
    try:
        encrypted_obj = s3.get_object(Bucket=bucket_name, Key=s3_key)
        encrypted_content = encrypted_obj['Body'].read()
    except ClientError as e:
        print(f"Failed to retrieve encrypted file at {s3_key}: {e}")
        return

    # Decrypt the file content
    aesgcm = AESGCM(key)
    try:
        decrypted_content = aesgcm.decrypt(nonce, encrypted_content, aad)
        print(f"Decryption successful for {s3_key}")
    except Exception as e:
        print(f"Decryption failed: {e}")
        return

    # Save decrypted content to a temporary file path or handle it as needed
    temp_file_path = f"/tmp/{s3_key.split('/')[-1]}"
    with open(temp_file_path, 'wb') as f:
        f.write(decrypted_content)
    print(f"Decrypted content saved to {temp_file_path}")

    # Return the path to the decrypted file
    return temp_file_path

In [35]:
bucket_name = 'otg-fraud-dataml-prod'
s3_key = 'acm/daily/year=2024/month=10/day=29/20241030131824_acm_output.csv'
key=b')\xd9;_kp\xd5\x9dY\xc6\xb2UD\xa4r\xe1}\xf3\xb0+C\xba\xb4W\xa1\xc8\x00m\x94\xe2y\xc2'
aad = b"authenticated additional data" 

In [36]:
decrypted_file=decrypt_existing_encrypted_file(s3_key, bucket_name, key, aad)

Successfully retrieved nonce from acm/daily/year=2024/month=10/day=29/20241030131824_acm_output_info.json
Decryption successful for acm/daily/year=2024/month=10/day=29/20241030131824_acm_output.csv
Decrypted content saved to /tmp/20241030131824_acm_output.csv


In [26]:
date_part = s3_key.split('/') # ['2024', '10', '2']
timestamp = date_part[-1].split('_')[0]
# Extract year, month, and day from the file path
date_part = s3_key.split('/')[-4:-1]  # ['2024', '10', '2']

year, month, day = date_part[0], date_part[1], date_part[2]

# Construct the S3 path
s3_output_path = f"s3://otg-fraud-dataml-prod/acm/daily/temporary/{year}/{month}/{day}/{timestamp}_acm_output.csv"

In [27]:
s3_output_path

's3://otg-fraud-dataml-prod/acm/daily/temporary/year=2024/month=10/day=29/20241030131824_acm_output.csv'

In [2]:
import boto3
import logging
import warnings
import pandas as pd
from sys import stderr

In [3]:
s3 = boto3.client('s3')
warnings.filterwarnings("ignore")
logging.basicConfig(stream=stderr, level="INFO")

In [13]:
import os
import boto3
from io import BytesIO
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from io import BytesIO
import json

In [34]:

def decrypt_existing_encrypted_file(s3_key, bucket_name, key, aad):
    s3 = boto3.client('s3')

    # Define the path to the existing info.json file
    info_file_key = s3_key.replace('.csv', '_info.json')

    # Retrieve the existing info.json file
    try:
        info_obj = s3.get_object(Bucket=bucket_name, Key=info_file_key)
        info_data = json.loads(info_obj['Body'].read())
        nonce = bytes.fromhex(info_data['nonce'])  # Convert hex nonce back to bytes
        print(f"Successfully retrieved nonce from {info_file_key}")
    except ClientError as e:
        print(f"Failed to retrieve info.json at {info_file_key}: {e}")
        return

    # Fetch the encrypted file from S3
    try:
        encrypted_obj = s3.get_object(Bucket=bucket_name, Key=s3_key)
        encrypted_content = encrypted_obj['Body'].read()
    except ClientError as e:
        print(f"Failed to retrieve encrypted file at {s3_key}: {e}")
        return

    # Decrypt the file content
    aesgcm = AESGCM(key)
    try:
        decrypted_content = aesgcm.decrypt(nonce, encrypted_content, aad)
        print(f"Decryption successful for {s3_key}")
    except Exception as e:
        print(f"Decryption failed: {e}")
        return

    # Save decrypted content to a temporary file path or handle it as needed
    temp_file_path = f"/tmp/{s3_key.split('/')[-1]}"
    with open(temp_file_path, 'wb') as f:
        f.write(decrypted_content)
    print(f"Decrypted content saved to {temp_file_path}")

    # Return the path to the decrypted file
    return temp_file_path

In [51]:
bucket_name = 'otg-fraud-dataml-prod'
s3_path = 's3://otg-fraud-dataml-prod/acm/daily/year=2024/month=10/day=29/20241030131824_acm_output.csv'
key=b')\xd9;_kp\xd5\x9dY\xc6\xb2UD\xa4r\xe1}\xf3\xb0+C\xba\xb4W\xa1\xc8\x00m\x94\xe2y\xc2'
aad = b"authenticated additional data" 

In [52]:
decrypted_file=decrypt_existing_encrypted_file(s3_path)

Successfully retrieved nonce from acm/daily/year=2024/month=10/day=29/20241030131824_acm_output_info.json
Decryption successful for acm/daily/year=2024/month=10/day=29/20241030131824_acm_output.csv
Decrypted content uploaded back to S3 at s3://otg-fraud-dataml-prod/acm/daily/temporary/year=2024/month=10/day=29/20241030131824_acm_output.csv


In [36]:
decrypted_file=decrypt_existing_encrypted_file(s3_key, bucket_name, key, aad)

Successfully retrieved nonce from acm/daily/year=2024/month=10/day=29/20241030131824_acm_output_info.json
Decryption successful for acm/daily/year=2024/month=10/day=29/20241030131824_acm_output.csv
Decrypted content saved to /tmp/20241030131824_acm_output.csv


In [26]:
date_part = s3_key.split('/') # ['2024', '10', '2']
timestamp = date_part[-1].split('_')[0]
# Extract year, month, and day from the file path
date_part = s3_key.split('/')[-4:-1]  # ['2024', '10', '2']

year, month, day = date_part[0], date_part[1], date_part[2]

# Construct the S3 path
s3_output_path = f"s3://otg-fraud-dataml-prod/acm/daily/temporary/{year}/{month}/{day}/{timestamp}_acm_output.csv"

In [53]:
s3_output_path

's3://otg-fraud-dataml-prod/acm/daily/temporary/year=2024/month=10/day=29/20241030131824_acm_output.csv'

In [14]:
def extract_s3_key(s3_value):
    """Extract the S3 key if the full S3 path is provided, otherwise assume it's already a key."""
    if s3_value.startswith('s3://'):
        # Extract the key from the full S3 path
        return '/'.join(s3_value.split('/')[3:])
    else:
        # It's already a key
        return s3_value

In [38]:
extract_s3_key('s3://otg-fraud-dataml-prod/acm/daily/temporary/year=2024/month=10/day=29/20241030131824_acm_output.csv')

'acm/daily/temporary/year=2024/month=10/day=29/20241030131824_acm_output.csv'

In [48]:
def decrypt_existing_encrypted_file(s3_path):
    s3 = boto3.client('s3')
    s3_key = extract_s3_key(s3_path)
    info_file_key = s3_key.replace('.csv', '_info.json')

    # Retrieve the existing info.json file
    try:
        info_obj = s3.get_object(Bucket=bucket_name, Key=info_file_key)
        info_data = json.loads(info_obj['Body'].read())
        nonce = bytes.fromhex(info_data['nonce'])  # Convert hex nonce back to bytes
        print(f"Successfully retrieved nonce from {info_file_key}")
    except ClientError as e:
        print(f"Failed to retrieve info.json at {info_file_key}: {e}")
        return

    # Fetch the encrypted file from S3
    try:
        encrypted_obj = s3.get_object(Bucket=bucket_name, Key=s3_key)
        encrypted_content = encrypted_obj['Body'].read()
    except ClientError as e:
        print(f"Failed to retrieve encrypted file at {s3_key}: {e}")
        return

    # Decrypt the file content
    aesgcm = AESGCM(key)
    try:
        decrypted_content = aesgcm.decrypt(nonce, encrypted_content, aad)
        print(f"Decryption successful for {s3_key}")
    except Exception as e:
        print(f"Decryption failed: {e}")
        return
        
    date_part = s3_path.split('/') # ['2024', '10', '2']
    timestamp = date_part[-1].split('_')[0]
    # Extract year, month, and day from the file path
    date_part = s3_path.split('/')[-4:-1]  # ['2024', '10', '2']
    
    year, month, day = date_part[0], date_part[1], date_part[2]
    
    # Construct the S3 path
    s3_output_path = f"s3://otg-fraud-dataml-prod/acm/daily/temporary/{year}/{month}/{day}/{timestamp}_acm_output.csv"
    # Upload the decrypted content back to S3
    try:
        s3.put_object(
            Bucket=bucket_name,
            Key=s3_output_path,  # Adjust the path if needed
            Body=decrypted_content
        )
        print(f"Decrypted content uploaded back to S3 at {s3_output_path}")
    except Exception as e:
        print(f"Failed to upload decrypted file: {e}")


In [56]:
from datetime import datetime

# Original string with time and microseconds
end_date = '2024-11-17 07:53:01.716069'

# Convert to datetime object using the full format
end_date_obj = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S.%f')

# Format to just the date part
end_date = end_date_obj.strftime('%Y-%m-%d')

print(end_date)  # Output: '2024-11-17'


2024-11-17


In [15]:
 payment_paths = ['s3://otg-fraud-dataml-prod/temporary/v_payment_details/daily/year=2024/month=11/day=18/20241121132517_lacm_000.csv', 's3://otg-fraud-dataml-prod/temporary/v_payment_details/daily/year=2024/month=11/day=19/20241121132517_lacm_000.csv', 's3://otg-fraud-dataml-prod/temporary/v_payment_details/daily/year=2024/month=11/day=20/20241121132517_lacm_000.csv']

In [21]:
order_paths= ['s3://otg-fraud-dataml-prod/temporary/v_order_details/daily/year=2024/month=11/day=18/20241121132517_lacm_000.csv', 's3://otg-fraud-dataml-prod/temporary/v_order_details/daily/year=2024/month=11/day=19/20241121132517_lacm_000.csv', 's3://otg-fraud-dataml-prod/temporary/v_order_details/daily/year=2024/month=11/day=20/20241121132517_lacm_000.csv']

In [22]:
keys_path=payment_paths + order_paths

In [19]:
# Extract S3 keys from full paths or use them directly if they are keys
s3_keys = [print(path) for path in keys_path]
print("s3_keys",s3_keys)

['s3://otg-fraud-dataml-prod/temporary/v_payment_details/daily/year=2024/month=11/day=18/20241121132517_lacm_000.csv', 's3://otg-fraud-dataml-prod/temporary/v_payment_details/daily/year=2024/month=11/day=19/20241121132517_lacm_000.csv', 's3://otg-fraud-dataml-prod/temporary/v_payment_details/daily/year=2024/month=11/day=20/20241121132517_lacm_000.csv']
s3_keys [None]


In [23]:
keys_path

['s3://otg-fraud-dataml-prod/temporary/v_payment_details/daily/year=2024/month=11/day=18/20241121132517_lacm_000.csv',
 's3://otg-fraud-dataml-prod/temporary/v_payment_details/daily/year=2024/month=11/day=19/20241121132517_lacm_000.csv',
 's3://otg-fraud-dataml-prod/temporary/v_payment_details/daily/year=2024/month=11/day=20/20241121132517_lacm_000.csv',
 's3://otg-fraud-dataml-prod/temporary/v_order_details/daily/year=2024/month=11/day=18/20241121132517_lacm_000.csv',
 's3://otg-fraud-dataml-prod/temporary/v_order_details/daily/year=2024/month=11/day=19/20241121132517_lacm_000.csv',
 's3://otg-fraud-dataml-prod/temporary/v_order_details/daily/year=2024/month=11/day=20/20241121132517_lacm_000.csv']